In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.stats import weibull_min, norm, gamma, lognorm
from scipy.integrate import solve_ivp
from scipy.signal import savgol_filter
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ----------------------------------------------------------------------------------------
# 1. Advanced Physical Modeling
# ----------------------------------------------------------------------------------------

class PumpComponent:
    """Base class for pump components."""
    def __init__(self, name):
        self.name = name
        self.state = {}

    def update(self, time, dt, *args, **kwargs):
        """Abstract method for updating the component's state."""
        raise NotImplementedError

    def get_state(self):
        """Returns the current state of the component."""
        return self.state

class Impeller(PumpComponent):
    """Models the impeller."""
    def __init__(self, name, design_diameter, material_strength, roughness_factor):
        super().__init__(name)
        self.design_diameter = design_diameter # in meter
        self.material_strength = material_strength # in MPa
        self.roughness_factor = roughness_factor # dimensionless
        self.state = {'wear_rate': 0, 'surface_roughness': 0, 'efficiency_loss': 0}

    def update(self, time, dt, flow_rate, fluid_density, rotational_speed):
        """Update the impeller state."""
        # Simplified wear model based on flow rate and rotational speed
        wear_rate = self.roughness_factor * flow_rate * rotational_speed * dt * 1e-6
        self.state['wear_rate'] += wear_rate
        self.state['surface_roughness'] = np.sqrt(self.state['wear_rate']) # increase surface roughness
        self.state['efficiency_loss'] = 0.01 * self.state['wear_rate'] # efficiency loss with wear
        return self.state

class Bearing(PumpComponent):
    """Models the bearing."""
    def __init__(self, name, design_load_capacity, lubrication_viscosity):
        super().__init__(name)
        self.design_load_capacity = design_load_capacity # in N
        self.lubrication_viscosity = lubrication_viscosity # in Pa.s
        self.state = {'temperature': 30, 'degradation': 0, 'vibration': 0}

    def update(self, time, dt, load, rotational_speed):
        """Update the bearing state."""
        # Simplified temperature model based on load and rotational speed
        friction_heat = load * rotational_speed * self.lubrication_viscosity * dt * 1e-6
        self.state['temperature'] += friction_heat
        self.state['degradation'] = 0.001 * self.state['temperature'] # degradation based on temperature
        self.state['vibration'] = 0.01 * self.state['temperature'] # vibration based on temperature
        return self.state

class Seal(PumpComponent):
    """Models the seal."""
    def __init__(self, name, design_pressure_limit, material_strength):
        super().__init__(name)
        self.design_pressure_limit = design_pressure_limit # in Pa
        self.material_strength = material_strength # in MPa
        self.state = {'leakage_rate': 0, 'degradation': 0}

    def update(self, time, dt, pressure, fluid_density):
        """Update the seal state."""
        # Simplified leakage model based on pressure
        leakage_rate = 0.0001 * pressure * dt * np.sqrt(fluid_density)
        self.state['leakage_rate'] += leakage_rate
        self.state['degradation'] = 0.0001 * self.state['leakage_rate'] # degradation based on leakage
        return self.state

class PumpSystem:
    """Models the overall pump system."""
    def __init__(self, design_flow, design_head, design_power, fluid_density=1000):
        self.design_flow = design_flow # in m3/h
        self.design_head = design_head # in meter
        self.design_power = design_power # in kW
        self.fluid_density = fluid_density # in kg/m3
        self.g = 9.81 # m/s2
        self.components = {}
        self.state = {'flow_rate': 0, 'head': 0, 'power': 0, 'efficiency': 0, 'pressure': 0}

        # Pump curve coefficients (H = ah^2 + bh + c)
        self.a = -self.design_head / (self.design_flow ** 2)
        self.b = 0
        self.c = self.design_head

        # System curve coefficient
        self.k = self.design_head / (self.design_flow ** 2)

    def add_component(self, component):
        """Add a component to the pump system."""
        self.components[component.name] = component

    def calculate_head(self, flow_rate):
        """Calculate pump head using pump curve."""
        return self.a * flow_rate**2 + self.b * flow_rate + self.c

    def calculate_system_head(self, flow_rate):
        """Calculate system head loss."""
        return self.k * flow_rate**2

    def calculate_power(self, flow_rate, head):
        """Calculate hydraulic power."""
        return (flow_rate * self.fluid_density * self.g * head) / (3.6e6) # in kW

    def calculate_motor_current(self, power, voltage=400, motor_efficiency=0.85):
        """Calculate motor current based on power and efficiency."""
        return (power * 1000) / (np.sqrt(3) * voltage * motor_efficiency * 0.85) # in Amperes

    def update_state(self, time, dt, flow_rate, rotational_speed):
        """Update the overall state of the pump system."""
        head = self.calculate_head(flow_rate)
        power = self.calculate_power(flow_rate, head)
        self.state['flow_rate'] = flow_rate
        self.state['head'] = head
        self.state['power'] = power
        self.state['efficiency'] = (power / self.design_power) if self.design_power > 0 else 0
        self.state['pressure'] = head * self.fluid_density * self.g / 100000 # in bar

        # Update all components
        for component in self.components.values():
            if isinstance(component, Impeller):
                component.update(time, dt, flow_rate, self.fluid_density, rotational_speed)
            elif isinstance(component, Bearing):
                component.update(time, dt, self.state['power'] * 1000, rotational_speed)
            elif isinstance(component, Seal):
                component.update(time, dt, self.state['pressure'] * 100000, self.fluid_density)

        return self.state

# ----------------------------------------------------------------------------------------
# 2. Advanced Degradation Modeling
# ----------------------------------------------------------------------------------------

class DegradationModel:
    """Base class for degradation models."""
    def __init__(self, name):
        self.name = name

    def predict_degradation(self, time, operating_conditions):
        """Abstract method for predicting degradation."""
        raise NotImplementedError

class PhysicsBasedDegradation(DegradationModel):
    """Models degradation based on physical processes."""
    def __init__(self, name, degradation_rate_factor):
        super().__init__(name)
        self.degradation_rate_factor = degradation_rate_factor

    def predict_degradation(self, time, operating_conditions):
        """Predict degradation based on time and operating conditions."""
        # Example: Degradation increases with time and operating load
        load = operating_conditions.get('load', 1)
        return self.degradation_rate_factor * time * load

class MLBasedDegradation(DegradationModel):
    """Models degradation using a machine learning model."""
    def __init__(self, name, model_params):
        super().__init__(name)
        self.model = MLPRegressor(**model_params)
        self.scaler = StandardScaler()
        self.is_fitted = False

    def train(self, X, y):
        """Train the machine learning model."""
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
        self.is_fitted = True

    def predict_degradation(self, time, operating_conditions):
        """Predict degradation using the trained machine learning model."""
        X = np.array([list(operating_conditions.values())])
        if self.is_fitted:
            X_scaled = self.scaler.transform(X)
            return self.model.predict(X_scaled)[0]
        else:
            return 0 # return 0 if the model is not fitted

# ----------------------------------------------------------------------------------------
# 3. Realistic Failure Modeling
# ----------------------------------------------------------------------------------------

class FailureModel:
    """Base class for failure models."""
    def __init__(self, name):
        self.name = name

    def predict_failure(self, time, operating_conditions, component_state):
        """Abstract method for predicting failure."""
        raise NotImplementedError

class MultiFactorFailure(FailureModel):
    """Models failure based on multiple interacting factors."""
    def __init__(self, name, stress_threshold, fatigue_threshold, temperature_threshold):
        super().__init__(name)
        self.stress_threshold = stress_threshold
        self.fatigue_threshold = fatigue_threshold
        self.temperature_threshold = temperature_threshold

    def predict_failure(self, time, operating_conditions, component_state):
        """Predict failure based on stress, fatigue, and temperature."""
        stress = operating_conditions.get('stress', 0)
        fatigue = operating_conditions.get('fatigue', 0)
        temperature = component_state.get('temperature', 0)

        failure_probability = (stress > self.stress_threshold) + \
                              (fatigue > self.fatigue_threshold) + \
                              (temperature > self.temperature_threshold)

        return failure_probability > 0

class FailurePropagation:
    """Models how a failure in one component can propagate to others."""
    def __init__(self, name, propagation_matrix):
        self.name = name
        self.propagation_matrix = propagation_matrix

    def propagate(self, failed_component, component_states):
        """Propagate failure to other components."""
        if failed_component not in self.propagation_matrix:
            return {}

        propagated_failures = {}
        for target_component, probability in self.propagation_matrix[failed_component].items():
            if np.random.rand() < probability:
                propagated_failures[target_component] = True
        return propagated_failures

# ----------------------------------------------------------------------------------------
# 4. External Factor Modeling
# ----------------------------------------------------------------------------------------

class ExternalFactorModel:
    """Base class for external factor models."""
    def __init__(self, name):
        self.name = name

    def get_external_conditions(self, time):
        """Abstract method for getting external conditions."""
        raise NotImplementedError

class EnvironmentalFactor(ExternalFactorModel):
    """Models the impact of environmental factors."""
    def __init__(self, name, base_temperature, temperature_amplitude, humidity_amplitude):
        super().__init__(name)
        self.base_temperature = base_temperature
        self.temperature_amplitude = temperature_amplitude
        self.humidity_amplitude = humidity_amplitude

    def get_external_conditions(self, time):
        """Get environmental conditions based on time."""
        temperature = self.base_temperature + self.temperature_amplitude * np.sin(2 * np.pi * time / (365 * 24))
        humidity = self.humidity_amplitude * np.cos(2 * np.pi * time / (365 * 24))
        return {'ambient_temperature': temperature, 'humidity': humidity}

class OperationalFactor(ExternalFactorModel):
    """Models the impact of operational variability."""
    def __init__(self, name, base_flow, flow_amplitude):
        super().__init__(name)
        self.base_flow = base_flow
        self.flow_amplitude = flow_amplitude

    def get_external_conditions(self, time):
        """Get operational conditions based on time."""
        flow_variation = self.flow_amplitude * np.sin(2 * np.pi * time / 24)
        return {'flow_rate_variation': flow_variation}

# ----------------------------------------------------------------------------------------
# 5. Generative Models
# ----------------------------------------------------------------------------------------

class GAN:
    """Generative Adversarial Network for time series."""
    def __init__(self, input_shape, latent_dim=100):
        self.input_shape = input_shape
        self.latent_dim = latent_dim
        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()
        self.gan = self._build_gan()

    def _build_generator(self):
        generator = keras.Sequential([
            layers.Dense(128, input_dim=self.latent_dim),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(256),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(np.prod(self.input_shape), activation='tanh'),
            layers.Reshape(self.input_shape)
        ])
        return generator

    def _build_discriminator(self):
        discriminator = keras.Sequential([
            layers.Flatten(input_shape=self.input_shape),
            layers.Dense(256),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(128),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(1, activation='sigmoid')
        ])
        return discriminator

    def _build_gan(self):
        self.discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.discriminator.trainable = False
        gan_input = keras.Input(shape=(self.latent_dim,))
        gan_output = self.discriminator(self.generator(gan_input))
        gan = keras.Model(gan_input, gan_output)
        gan.compile(loss='binary_crossentropy', optimizer='adam') # added loss here
        return gan

    def train(self, X, epochs=1000, batch_size=32):
        X = np.array(X)
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X.shape[0], batch_size)
            real_data = X[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_data = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(real_data, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_data, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.gan.train_on_batch(noise, real)

            if epoch % 100 == 0:
                print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    def generate(self, num_samples):
        noise = np.random.normal(0, 1, (num_samples, self.latent_dim))
        generated_data = self.generator.predict(noise)
        return generated_data

class VAE:
    """Variational Autoencoder for feature generation."""
    def __init__(self, input_shape, latent_dim=10):
        self.input_shape = input_shape
        self.latent_dim = latent_dim
        self.encoder = self._build_encoder()
        self.decoder = self._build_decoder()
        self.vae = self._build_vae()

    def _build_encoder(self):
        encoder_inputs = keras.Input(shape=self.input_shape)
        x = layers.Flatten()(encoder_inputs)
        x = layers.Dense(128, activation='relu')(x)
        z_mean = layers.Dense(self.latent_dim)(x)
        z_log_var = layers.Dense(self.latent_dim)(x)
        z = layers.Lambda(self._sampling)([z_mean, z_log_var])
        return keras.Model(encoder_inputs, [z_mean, z_log_var, z])

    def _build_decoder(self):
        latent_inputs = keras.Input(shape=(self.latent_dim,))
        x = layers.Dense(128, activation='relu')(latent_inputs)
        x = layers.Dense(np.prod(self.input_shape), activation='sigmoid')(x)
        decoder_outputs = layers.Reshape(self.input_shape)(x)
        return keras.Model(latent_inputs, decoder_outputs)

    def _sampling(self, args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    # New Keras Layer for KL Loss calculation
    class KLLossLayer(layers.Layer):
        def call(self, z_mean, z_log_var):
            kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
            self.add_loss(tf.reduce_mean(kl_loss))
            return kl_loss

    def _build_vae(self):
        vae_inputs = keras.Input(shape=self.input_shape)
        z_mean, z_log_var, z = self.encoder(vae_inputs)
        vae_outputs = self.decoder(z)
        vae = keras.Model(vae_inputs, vae_outputs)
        # Use the new KLLossLayer
        self.KLLossLayer()(z_mean, z_log_var) # add the loss
        vae.compile(optimizer='adam', loss='mse') # added loss here
        return vae

    def train(self, X, epochs=100, batch_size=32):
        X = np.array(X)
        self.vae.fit(X, X, epochs=epochs, batch_size=batch_size, verbose=1)

    def generate(self, num_samples):
        z = np.random.normal(0, 1, size=(num_samples, self.latent_dim))
        generated_data = self.decoder.predict(z)
        return generated_data

# ----------------------------------------------------------------------------------------
# 6. Data Generation Function
# ----------------------------------------------------------------------------------------

def generate_maximum_pump_data(num_pumps, start_date, num_years=5):
    """
    Generates high-fidelity synthetic pump data using advanced models.
    """
    np.random.seed(42)
    data = []
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    hours_per_year = 365 * 24
    num_time_steps = int(num_years * hours_per_year)

    for pump_id in range(1, num_pumps + 1):
        # Initialize pump system with random design points
        design_flow = np.random.normal(100, 5)
        design_head = np.random.normal(50, 2)
        design_power = np.random.normal(30, 1)
        pump_system = PumpSystem(design_flow, design_head, design_power)

        # Initialize components
        impeller = Impeller("impeller", design_diameter=0.2, material_strength=200, roughness_factor=0.001)
        bearing = Bearing("bearing", design_load_capacity=10000, lubrication_viscosity=0.01)
        seal = Seal("seal", design_pressure_limit=1000000, material_strength=100)
        pump_system.add_component(impeller)
        pump_system.add_component(bearing)
        pump_system.add_component(seal)

        # Initialize degradation models
        physics_degradation = PhysicsBasedDegradation("physics_degradation", degradation_rate_factor=0.00001)
        ml_degradation = MLBasedDegradation("ml_degradation", model_params={'hidden_layer_sizes': (100, 50), 'activation': 'relu', 'solver': 'adam', 'max_iter': 100})

        # Initialize failure models
        multi_factor_failure = MultiFactorFailure("multi_factor_failure", stress_threshold=100, fatigue_threshold=100, temperature_threshold=100)

        # Initialize failure propagation
        propagation_matrix = {
            "impeller": {"bearing": 0.3, "seal": 0.2},
            "bearing": {"impeller": 0.1, "seal": 0.2},
            "seal": {"bearing": 0.1, "impeller": 0.1}
        }
        failure_propagation = FailurePropagation("failure_propagation", propagation_matrix)

        # Initialize external factor models
        environmental_factor = EnvironmentalFactor("environmental_factor", base_temperature=25, temperature_amplitude=10, humidity_amplitude=0.2)
        operational_factor = OperationalFactor("operational_factor", base_flow=design_flow, flow_amplitude=0.1)

        # Initialize GAN for sensor data generation
        gan_input_shape = (10, 1) # 10 timesteps, 1 variable
        gan = GAN(input_shape=gan_input_shape, latent_dim=100)

        # Initialize VAE for feature generation
        vae_input_shape = (10,) # 10 features
        vae = VAE(input_shape=vae_input_shape, latent_dim=10)

        # Generate base operating conditions
        time_vector = np.arange(num_time_steps)
        flow_rate_variation = operational_factor.get_external_conditions(time_vector)['flow_rate_variation']
        base_flow = design_flow * (1 + flow_rate_variation)
        rotational_speed = 1000 # RPM

        # Initialize data lists
        pump_data = []
        failure_events = []
        maintenance_events = []

        # Generate training data for ML degradation model
        training_data_size = min(1000, num_time_steps)
        training_X = []
        training_y = []
        for t in range(training_data_size):
            external_conditions = environmental_factor.get_external_conditions(t)
            operating_conditions = {'load': pump_system.state.get('power', 1), 'stress': pump_system.state.get('pressure', 0) , 'fatigue': bearing.state.get('degradation', 0), 'temperature': bearing.state.get('temperature', 0)}
            training_X.append(list(operating_conditions.values()))
            training_y.append(physics_degradation.predict_degradation(t, operating_conditions))
        ml_degradation.train(np.array(training_X), np.array(training_y))

        # Time loop
        for t in range(num_time_steps):
            # Get external conditions
            external_conditions = environmental_factor.get_external_conditions(t)
            operating_conditions = {'load': pump_system.state.get('power', 1), 'stress': pump_system.state.get('pressure', 0) , 'fatigue': bearing.state.get('degradation', 0), 'temperature': bearing.state.get('temperature', 0)}

            # Update system state
            pump_state = pump_system.update_state(t, 1, base_flow[t], rotational_speed)

            # Predict degradation using physics and machine learning models
            physics_degradation_value = physics_degradation.predict_degradation(t, operating_conditions)
            ml_degradation_value = ml_degradation.predict_degradation(t, operating_conditions)

            # Predict failure
            failure_predicted = False
            for component_name, component in pump_system.components.items():
                failure_predicted = multi_factor_failure.predict_failure(t, operating_conditions, component.get_state())
                if failure_predicted:
                    failure_events.append({'time': t, 'component': component_name})
                    # Propagate failure
                    propagated_failures = failure_propagation.propagate(component_name, pump_system.components)
                    for target_component in propagated_failures:
                        failure_events.append({'time': t, 'component': target_component})
                    break

            # Maintenance Simulation
            if t % 4000 == 0: # schedule maintenance every 4000 hours
                maintenance_events.append({'time': t})
                for component in pump_system.components.values():
                    if isinstance(component, Bearing):
                        component.state['temperature'] *= 0.8
                        component.state['vibration'] *= 0.8
                    if isinstance(component, Impeller):
                        component.state['efficiency_loss'] *= 0.5


            # Collect data for GAN and VAE training
            if t >= 10:
                gan_data = []
                for i in range(10):
                    gan_data.append([pump_system.state['flow_rate']])
                gan_data = np.array(gan_data)
                gan_data = np.reshape(gan_data, gan_input_shape)
                vae_data = [pump_system.state['flow_rate'], pump_system.state['head'], pump_system.state['power'],
                            pump_system.state['efficiency'], pump_system.state['pressure'],
                            bearing.state['temperature'], bearing.state['vibration'],
                            impeller.state['wear_rate'], impeller.state['surface_roughness'],
                            seal.state['leakage_rate']]
                if t % 1000 == 0:
                    gan.train([gan_data], epochs=10, batch_size=1)
                    vae.train([vae_data], epochs=10, batch_size=1)

            # Assemble data record
            record = {
                'pump_id': pump_id,
                'timestamp': start_datetime + timedelta(hours=t),
                'flow_rate': pump_system.state['flow_rate'],
                'discharge_pressure': pump_system.state['pressure'],
                'motor_current': pump_system.calculate_motor_current(pump_system.state['power']),
                'motor_vibration': bearing.state['vibration'],
                'bearing_temperature': bearing.state['temperature'],
                'impeller_wear_rate': impeller.state['wear_rate'],
                'impeller_surface_roughness': impeller.state['surface_roughness'],
                'impeller_efficiency_loss': impeller.state['efficiency_loss'],
                'seal_leakage_rate': seal.state['leakage_rate'],
                'power': pump_system.state['power'],
                'efficiency': pump_system.state['efficiency'],
                'ambient_temperature': external_conditions['ambient_temperature'],
                'humidity': external_conditions['humidity'],
                'degradation_physics': physics_degradation_value,
                'degradation_ml': ml_degradation_value,
                'failure_predicted': failure_predicted,
                'maintenance_event': t in [event['time'] for event in maintenance_events],
            }
            pump_data.append(record)

        data.extend(pump_data)

    df = pd.DataFrame(data)
    return df

# Generate synthetic data
if __name__ == "__main__":
    synthetic_data = generate_maximum_pump_data(
        num_pumps=50,
        start_date="2020-01-01",
        num_years=1
    )
    synthetic_data.to_csv("maximum_synthetic_pump_data.csv", index=False)
    print(f"Generated {len(synthetic_data)} records of synthetic pump data")

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
import pandas as pd
import numpy as np

def correct_negative_values(file_path):
    """
    Corrects negative values in specified columns of a CSV file using Pandas.

    Parameters:
        file_path (str): The path to the CSV file.
    """

    try:
        # Load the CSV file into a Pandas DataFrame
        df = pd.read_csv("C:\Coding\Project\Predictive Maintenance Centrifugal Pumps\predictive-maintenance-centrifugal-pumps\data\maximum_synthetic_pump_data.csv")

        # Columns to correct
        columns_to_correct = ['discharge_pressure', 'motor_current', 'power', 'efficiency', 'seal_leakage_rate']

        # Correct negative values using absolute value
        for column in columns_to_correct:
            if column in df.columns:
                df[column] = df[column].abs()

        # Correct the duplicate degradatic column
        if 'degradatic' in df.columns:
            df.rename(columns={'degradatic': 'degradation_physics'}, inplace=True)
        if 'degradatic.1' in df.columns:
            df.rename(columns={'degradatic.1': 'degradation_ml'}, inplace=True)

        # Save the modified DataFrame back to the same CSV file
        df.to_csv(file_path, index=False)

        print(f"Negative values in {columns_to_correct} columns have been corrected in '{file_path}'.")

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the path to your CSV file
file_path = "maximum_synthetic_pump_data.csv"

# Call the function to correct the negative values
correct_negative_values(file_path)

<>:14: SyntaxWarning: invalid escape sequence '\C'
<>:14: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Raihan\AppData\Local\Temp\ipykernel_1716\2348458541.py:14: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_csv("C:\Coding\Project\Predictive Maintenance Centrifugal Pumps\predictive-maintenance-centrifugal-pumps\data\maximum_synthetic_pump_data.csv")


Negative values in ['discharge_pressure', 'motor_current', 'power', 'efficiency', 'seal_leakage_rate'] columns have been corrected in 'maximum_synthetic_pump_data.csv'.
